In [1]:
import sys, time
import numpy as np
import arkouda as ak

In [2]:
ak.connect(server="localhost", port=5555)

4.3.1
psp =  tcp://localhost:5555
connected to tcp://localhost:5555


In [3]:
# files and locations on my(mike's) laptop
hdffiles = ["/Users/valton/lanl_netflow/hdf5/netflow_day-02-00.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-01.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-02.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-03.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-04.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-05.hdf"]

In [4]:
fields = ['srcIP', 'dstIP', 'srcPort', 'dstPort', 'start']

In [5]:
%time nfDF = {field: ak.read_hdf(field, hdffiles) for field in fields}

CPU times: user 1.7 ms, sys: 1.74 ms, total: 3.44 ms
Wall time: 4.9 s


In [6]:
print(nfDF['start'])
print(nfDF['srcIP'])
print(nfDF['dstIP'])
print(nfDF['srcPort'])
print(nfDF['dstPort'])

[118781 118783 118785 ... 140369 140369 140369]
[34111590301 34111676798 48764590996 ... 34112191431 34112191431 34112191431]
[34111773101 34112055194 34111367844 ... 34111545319 31712083968 31712083968]
[5507 3137 5060 ... 78620 44270 49614]
[46272 445 5060 ... 95765 80 80]


In [7]:
print(nfDF)

{'srcIP': array([34111590301, 34111676798, 48764590996, ..., 34112191431, 34112191431, 34112191431]), 'dstIP': array([34111773101, 34112055194, 34111367844, ..., 34111545319, 31712083968, 31712083968]), 'srcPort': array([5507, 3137, 5060, ..., 78620, 44270, 49614]), 'dstPort': array([46272, 445, 5060, ..., 95765, 80, 80]), 'start': array([118781, 118783, 118785, ..., 140369, 140369, 140369])}


In [8]:
print(ak.info(ak.AllSymbols))
print(nfDF['srcIP'].name)

name:"id_5" dtype:"int64" size:60000000 ndim:1 shape:(60000000) itemsize:8
name:"id_4" dtype:"int64" size:60000000 ndim:1 shape:(60000000) itemsize:8
name:"id_3" dtype:"int64" size:60000000 ndim:1 shape:(60000000) itemsize:8
name:"id_2" dtype:"int64" size:60000000 ndim:1 shape:(60000000) itemsize:8
name:"id_1" dtype:"int64" size:60000000 ndim:1 shape:(60000000) itemsize:8

id_1


In [ ]:
%time u,c = ak.unique(nfDF['srcIP'], return_counts=True)

In [ ]:
print(u.size,u)
print(c.size,c)

In [ ]:
%time u,c = ak.unique(nfDF['dstIP'], return_counts=True)

In [ ]:
print(u.size,u)
print(c.size,c)

In [ ]:
%time u,c = ak.unique(nfDF['srcPort'], return_counts=True)

In [ ]:
print(u.size,u)
print(c.size,c)

In [ ]:
%time u,c = ak.unique(nfDF['dstPort'], return_counts=True)

In [ ]:
print(u.size,u)
print(c.size,c)

In [ ]:
print((nfDF['srcPort'] == 80).sum())
print((nfDF['dstPort'] == 80).sum())

In [ ]:
s = u[0:100] # pick off some dest ports
print(s)
%time t = ak.in1d(nfDF['dstPort'], s) # bool vector of where they are found in the data
print(t.sum())

In [ ]:
s = u[0:100].to_ndarray() # pick off some dest ports
dp = nfDF['dstPort'].to_ndarray()
print(s)
%time t = np.in1d(dp, s) # bool vector of where they are found in the data
print(t.sum())

In [ ]:
a = nfDF['start']
aMin = a.min()
aMax = a.max()
print(aMin,aMax, aMax - aMin)
%time h = ak.histogram(a,bins=100)
print(h)
print(h.sum(), a.size)

In [9]:
a = nfDF['start']
print(a.size, a)
print(ak.is_sorted(a))
%time iv = ak.argsort(a)
print(iv.size, iv)
b = a[iv]
print(b.size, b)
print(ak.is_sorted(b))

60000000 [118781 118783 118785 ... 140369 140369 140369]
True
CPU times: user 2.32 ms, sys: 731 µs, total: 3.05 ms
Wall time: 1min 7s
60000000 [0 1 2 ... 59999997 59999998 59999999]
60000000 [118781 118783 118785 ... 140369 140369 140369]
True


In [ ]:
a = nfDF['dstPort']
print(a.size, a)
print(ak.is_sorted(a))
%time iv = ak.argsort(a)
print(iv.size, iv)
b = a[iv]
print(b.size, b)
print(ak.is_sorted(b))

60000000 [46272 445 5060 ... 95765 80 80]
False


In [ ]:
#ak.shutdown()
#ak.disconnect()